In [ ]:
# !pip install nltk
# !pip install datasets
!pip install transformers[torch]
!pip install tokenizers
!pip install evaluate
!pip install rouge_score
!pip install sentencepiece
!pip install huggingface_hub!

ERROR: Invalid requirement: 'huggingface_hub!'


In [ ]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/finetuned_flant5.zip

Archive:  /content/drive/MyDrive/finetuned_flant5.zip
   creating: content/results/checkpoint-1000/
  inflating: content/results/checkpoint-1000/config.json  
  inflating: content/results/checkpoint-1000/model.safetensors  
  inflating: content/results/checkpoint-1000/added_tokens.json  
  inflating: content/results/checkpoint-1000/trainer_state.json  
  inflating: content/results/checkpoint-1000/optimizer.pt  
  inflating: content/results/checkpoint-1000/spiece.model  
  inflating: content/results/checkpoint-1000/training_args.bin  
  inflating: content/results/checkpoint-1000/special_tokens_map.json  
  inflating: content/results/checkpoint-1000/tokenizer_config.json  
  inflating: content/results/checkpoint-1000/scheduler.pt  
  inflating: content/results/checkpoint-1000/generation_config.json  
  inflating: content/results/checkpoint-1000/rng_state.pth  


In [ ]:
last_checkpoint = "/content/content/results/checkpoint-1000"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
my_question = "List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020."
inputs = "Please answer to this question: " + my_question

inputs = tokenizer.encode(inputs,return_tensors="pt")
outputs = finetuned_model.generate(
    inputs,
    max_length=100,  # Increase this value if outputs are incomplete
    temperature=0.0000001,
    do_sample=True, # Adjust temperature for diversity
    # top_k=50,        # Adjust top-k sampling
    # top_p=0.95,      # Adjust top-p sampling
    repetition_penalty=2.0  # Apply repetition penalty
)
answer = tokenizer.decode(outputs[0])
from textwrap import fill

print(answer)

<pad> Minister of Foreign Affairs (2010, 2011, 2012, 2013, 2014) President of the Russian Federation (2014, 2015, 2016, 2017, 2018, 2019, 2020)</s>


In [ ]:
!pip install sentence_transformers

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity



def read_QA_data(file):
    with open(file) as f:
        data = json.load(f)

    return data

class KnnSearch:
    def __init__(self,data=None, num_trees=None,emb_dim=None):
        self.num_trees=num_trees
        self.emb_dim=emb_dim
    def get_embeddings_for_data(self, data_ls):
        model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
        embeddings = model.encode(data_ls)
        return embeddings

    def get_top_n_neighbours(self,sentence,data_emb,train_data,k):
        sent_emb = self.get_embeddings_for_data(sentence)
        #data_emb = self.get_embeddings_for_data(transfer_questions)
        top_questions = []

        print("new_emb", sent_emb.shape, data_emb.shape)
        text_sims = cosine_similarity(data_emb,[sent_emb]).tolist()
        results_sims = zip(range(len(text_sims)), text_sims)
        sorted_similarities = sorted(results_sims, key=lambda x: x[1], reverse=True)

        #print("text_sims",sorted_similarities[:2])
        for idx, item in sorted_similarities[:k]:
                #if item[0] > 0.45:
                    top_questions.append(train_data[idx])

        return top_questions

def get_questions(train_data):
    train_questions = []
    for index, data in enumerate(train_data):
        train_questions.append(data["question"])
    return train_questions

def get_prompts(exemplars):
    prompt = ""
    for index, data in enumerate(exemplars):
        # prompt+="[Question {}]: ".format(index+1)+ data["question"]+"\n"
        prompt+="Example {}: ".format(index+1)+"\n"
        prompt+= data["question"]+"\n"
        answer = ""
        for ind, subans in enumerate(data["answers"]):
          answer+=subans
          if(ind+1 < len(data["answers"])):
            answer+=', '
        # prompt+="[Answer {}]: ".format(index+1)+ answer+"\n"
        prompt+= answer+"\n"

    print("prompt§§§§",prompt)
    return prompt

train_data = read_QA_data("train_TLQA.json")
test_data = read_QA_data("test_TLQA.json")

In [ ]:
knn_instance = KnnSearch()

train_questions = get_questions(train_data)
data_emb = knn_instance.get_embeddings_for_data(train_questions)

def few_shot_input(query, data_emb, train_data, k):
  exemplars = knn_instance.get_top_n_neighbours(query,data_emb,train_data,k)
  prompt_question = get_prompts(exemplars)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
!pip install langchain==0.1.9 --quiet

In [ ]:
import re
import string
from collections import Counter

def process_answer(answers):
  final_answer = ""
  for answer in answers:
    final_answer+= (answer+' ')

  return final_answer


def normalize_answer(s):
    """
    Lower text and remove punctuation, articles and extra whitespace.

    Args:
        s: String to normalize.

    Returns:
        Cleaned string with lowercase, no punctuations, no articles, and
            and extraneous whitespace.
    """

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    """Calculates F1 score.

    Args:
        prediction: Predicted answer span (string).
        ground_truth: True answer span (string).

    Returns:
        F1 score.
    """
    prediction_tokens = normalize_answer(prediction).split()

    ground_truth = process_answer(ground_truth)
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def exact_match_score(prediction, ground_truth):
    """Calculates exact match (EM) score.

    Args:
        prediction: Predicted answer span (string).
        ground_truth: True answer span (string).

    Returns:
        EM score.
    """
    ground_truth = process_answer(ground_truth)
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration
gen_cfg = GenerationConfig.from_pretrained(last_checkpoint)

In [ ]:
gen_cfg.max_new_tokens=512
gen_cfg.temperature=0.0000001 # 0.0
gen_cfg.return_full_text=True
gen_cfg.do_sample=True
gen_cfg.repetition_penalty=1.11

pipe = pipeline(task="text2text-generation", model=finetuned_model, tokenizer=tokenizer,
    generation_config=gen_cfg)
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from langchain.vectorstores.utils import filter_complex_metadata
from langchain.schema.document import Document

# from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import JSONLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def db_generate(text):
  # web_loader = JSONLoader(
  #     file_path=filename,
  #     jq_schema='.info',
  #     text_content=False)

  # web_doc = web_loader.load()
  # # print(web_doc)
  # updated_web_doc = filter_complex_metadata(web_doc)



  text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)
  # chunked_web_doc = text_splitter.split_documents(updated_web_doc)
  # print(len(chunked_web_doc))
  chunked_web_doc = [Document(page_content=x) for x in text_splitter.split_text(text)]

  embeddings = HuggingFaceEmbeddings()

  db_web = FAISS.from_documents(chunked_web_doc, embeddings)
  return db_web


In [ ]:
!pip install faiss-gpu==1.7.2 --quiet

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from tqdm import tqdm
from textwrap import fill
import json
from langchain.vectorstores import FAISS

# Ensure that your answer follows the same format as provided in the examples.
prompt_template = """
Please answer the question based on the context provided. Do not use any other information. If you can't find the relevant information in the context, just say you don't have enough information to answer the question. Don't try to make up an answer.
Context:
{context}{instruction}
Question: {question}
""".format(context = "{context}", instruction = "", question = "{question}")

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

EM_total = 0
F1_total = 0
with open('RAG.json') as f:
    d = json.load(f)
    RAG_data = d['info']


for i in tqdm(range(0,10)):
  query = test_data[i]['question']
  answers = test_data[i]['answers']
  # wikiid = test_data[i]['wikidata_ID']
  # crawler_summary(wikiid)
  db_web = db_generate(RAG_data[i])

  Chain_web = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type="stuff",
      # retriever=db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'k': 5, 'score_threshold': 0.8})
      # Similarity Search is the default way to retrieve documents relevant to a query, but we can use MMR by setting search_type = "mmr"
      # k defines how many documents are returned; defaults to 4.
      # score_threshold allows to set a minimum relevance for documents returned by the retriever, if we are using the "similarity_score_threshold" search type.
      # return_source_documents=True, # Optional parameter, returns the source documents used to answer the question
      retriever=db_web.as_retriever(search_kwargs={'k': 3}), # (search_kwargs={'k': 5, 'score_threshold': 0.8}),
      chain_type_kwargs={"prompt": prompt},
  )

  result = Chain_web.invoke(query)
  print(result)
  EM_total+=exact_match_score(result['result'], answers)
  F1_total+=f1_score(result['result'], answers)


  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
 10%|█         | 1/10 [00:35<05:23, 35.92s/it]

{'query': 'List all sports teams Anthony Grant, also known as Anthony Paul Shaun Andrew Daure Grant, played for from 2010 to 2020.', 'result': 'Chelsea F.C. (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018) Shrewsbury Town F.C. (2018, 2019, 2020)'}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
 20%|██        | 2/10 [01:09<04:38, 34.76s/it]

{'query': 'List all positions Oleksandr Turchynov, also known as Oleksandr Valentynovych Turchynov, held from 2010 to 2020.', 'result': 'President of Ukraine (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020) President of Ukraine (2020)'}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
 30%|███       | 3/10 [01:24<02:59, 25.63s/it]

...

{'query': 'List all coaches of Belarus national under-21 football team from 2010 to 2020', 'result': 'Andrei Zhukov (2010, 2011, 2012, 2013) Sergei Kostyan (2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)'}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
 90%|█████████ | 9/10 [05:01<00:31, 31.47s/it]

{'query': 'List all sports teams Joe Ledley, also known as Joseph Christopher Ledley, played for from 2010 to 2020.', 'result': 'Cardiff City F.C. (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017) England national under-17 football team (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020) England national under-21 football team (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020) England national under-21 football team (2010) England national association football team (2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020)'}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 10/10 [05:16<00:00, 31.64s/it]

{'query': 'List all political parties Laurențiu Rebega was a member of from 2010 to 2020.', 'result': 'Romanian Conservative Party (2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018) independent politician (2018, 2019, 2020)'}


In [ ]:
print('Exact Match: ', EM_total/len(test_data))
print('F1 Score: ', F1_total/10)

Exact Match:  0.0
F1 Score:  0.5969065412653884
